In [32]:
import scipy.ndimage.filters
import numpy as np
from skimage import feature
import glob
from PIL import Image

from sklearn.externals import joblib


In [33]:
# image to features
def feature_computation(image):
    features = []
    laplacian = scipy.ndimage.filters.laplace(image)
    lap_mat0 = np.matrix(laplacian[:,:,0])
    lap_mat1 = np.matrix(laplacian[:,:,1])
    lap_mat2 = np.matrix(laplacian[:,:,2])
    
    features.append(lap_mat0.sum())
    features.append(lap_mat1.sum())
    features.append(lap_mat2.sum())
    
    image_gray = image.convert('LA')
    gray_array = np.array(image_gray)[:,:,0]
    edges = feature.canny(gray_array,sigma = 3,low_threshold=5, high_threshold=20)
    max_val = np.amax(edges)

    
    for j in range(0,edges.shape[1]):
        for i in range(0,edges.shape[0]):
            if edges[i][j] > .1*max_val:
                edges[i][j] = 0
                if edges[i+1][j] < .1*max_val:
                    break
        for ii in range(0,edges.shape[0]):
            i = edges.shape[0] - ii -1
            if edges[i][j] > .1*max_val:
                edges[i][j] = 0
                if edges[i-1][j] < .1*max_val:
                    break
                    
    features.append(edges.sum())
    
    return features

    

    
    

In [34]:
def random_forest_predictor(image):
    feature_vec = feature_computation(image)
    model = joblib.load("radom_forest_model.pkl")
    
    return model.predict([feature_vec])
    

In [35]:
image_for_explanation=[]
for filename in glob.glob('./explain_images/*.png'):
    im=Image.open(filename)
    image_for_explanation.append(im.copy())
    im.close()
    
for im in image_for_explanation:
    print(str(feature_computation(im))+ "prediction: " + str(random_forest_predictor(im)) )
    
    
    

[2219517, 496822, 1392205, 48]prediction: [ True]
[1981178, 543742, 1726984, 19]prediction: [ True]
[2488116, 504349, 802735, 16]prediction: [ True]
[3056973, 688953, 1023098, 209]prediction: [ True]
[2945516, 566089, 645067, 117]prediction: [ True]
[2039076, 477923, 657913, 15]prediction: [ True]
[3469747, 847254, 825527, 503]prediction: [ True]
[3140536, 635418, 502830, 139]prediction: [ True]
[3715506, 465311, 511663, 232]prediction: [ True]
[2974939, 618256, 381461, 324]prediction: [ True]
[2596425, 647072, 727063, 278]prediction: [ True]
[3012269, 703842, 1765873, 81]prediction: [ True]
[3212288, 921969, 1337999, 516]prediction: [ True]
[1860288, 514271, 1263969, 229]prediction: [ True]
